## EZKL Jupyter Notebook Demo 

To get started install [miniconda](https://docs.conda.io/en/latest/miniconda.html) for your system. From there create an new evironment:

```bash
conda create -n ezkl python=3.9
```

Activate your newly created environment and install the requisite dependencies:

```bash
conda activate ezkl; pip install torch numpy ezkl;     
```

Now within this repository install the python bindings: 

```bash 

pip install -r requirements.txt
maturin develop --features python-bindings

```

💎 You should now be able to run the rest of the notebook 💎


In [1]:
# here we create and (potentially train a model)

from torch import nn
from ezkl import export
import ezkl_lib
import os
import json 


# Defines the model
# we got convs, we got relu, we got linear layers 
# What else could one want ???? 

class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()

        self.conv1 = nn.Conv2d(in_channels=1, out_channels=2, kernel_size=5, stride=2)
        self.conv2 = nn.Conv2d(in_channels=2, out_channels=3, kernel_size=5, stride=2)
        
        self.relu = nn.ReLU()

        self.d1 = nn.Linear(48, 48)
        self.d2 = nn.Linear(48, 10)

    def forward(self, x):
        # 32x1x28x28 => 32x32x26x26
        x = self.conv1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.relu(x)

        # flatten => 32 x (32*26*26)
        x = x.flatten(start_dim = 1)

        # 32 x (32*26*26) => 32x128
        x = self.d1(x)
        x = self.relu(x)

        # logits => 32x10
        logits = self.d2(x)
       
        return logits

circuit = MyModel()
export(circuit, input_shape = [1,28,28])



In [2]:

output_path = os.path.join('input.json')

data_path = os.path.join('input.json')

model_path = os.path.join('network.onnx')

res = ezkl_lib.forward(data_path, model_path, output_path)

with open(output_path, "r") as f:
    data = json.load(f)

In [3]:

# HERE WE SETUP THE CIRCUIT PARAMS 
# WE GOT KEYS 
# WE GOT CIRCUIT PARAMETERS 
# EVERYTHING ANYONE HAS EVER NEEDED FOR ZK 


pk_path = os.path.join('test.pk')
vk_path = os.path.join('test.vk')
circuit_params_path = os.path.join('circuit.params')
params_path = os.path.join('kzg.params')

res = ezkl_lib.setup(
        data_path,
        model_path,
        vk_path,
        pk_path,
        params_path,
        circuit_params_path,
    )

assert res == True
assert os.path.isfile(vk_path)
assert os.path.isfile(pk_path)
assert os.path.isfile(circuit_params_path)

assigning lookup input
assigning lookup output
laying out table for "RELU"
assigning lookup input
assigning lookup output
laying out table for "RELU"


In [4]:
# GENERATE A PROOF 
proof_path = os.path.join('test.pf')

res = ezkl_lib.prove(
        data_path,
        model_path,
        pk_path,
        proof_path,
        params_path,
        "poseidon",
        "single",
    )

assert res == True
assert os.path.isfile(proof_path)

assigning lookup input
assigning lookup output
assigning lookup input
assigning lookup output
laying out table for "RELU"
assigning lookup input
assigning lookup output
laying out table for "RELU"


In [5]:
# VERIFY IT 

res = ezkl_lib.verify(
        proof_path,
        circuit_params_path,
        vk_path,
        params_path,
        "poseidon",
    )

assert res == True
print("verified")

assigning lookup input
assigning lookup output


verified
